In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import os
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split

load_dotenv()

df = pd.read_csv('국토안전관리원_민원데이터_20250101.csv', encoding='cp949')

ModuleNotFoundError: No module named 'sklearn'

In [14]:
EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIM = 1536  # 사용 모델 차원에 맞게 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
COLLECTION_NAME = "complaints"  # 원하면 한글로 해도 됨

In [15]:
# OpenAI 클라이언트
oa_client = OpenAI(api_key=OPENAI_API_KEY)
chat_client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)

# Qdrant 클라이언트 (도커에서 localhost:6333 사용 중이라고 했으니까)
qdrant = QdrantClient(host="localhost", port=6333)

In [4]:
def get_embedding(text: str):
    """OpenAI 임베딩 생성 함수"""
    resp = oa_client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text
    )
    return resp.data[0].embedding

In [5]:
# 2) Qdrant 컬렉션 생성 (없으면)
collections = qdrant.get_collections().collections
existing_names = [c.name for c in collections]

if COLLECTION_NAME not in existing_names:
    qdrant.create_collection(
        collection_name="민원데이터",
        vectors_config=VectorParams(
            size=EMBEDDING_DIM,
            distance=Distance.COSINE
        )
    )

In [6]:
# 3) 배치 사이즈 설정
BATCH_SIZE = 128  # 64~256 사이 아무거나 편한 걸로

def get_embeddings_batch(texts):
    resp = oa_client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=texts,
    )
    # resp.data[i].embedding 리스트로 반환
    return [d.embedding for d in resp.data]


# 4) 배치로 처리
points_all = []
n = len(df)

for start in range(0, n, BATCH_SIZE):
    end = min(start + BATCH_SIZE, n)
    batch_df = df.iloc[start:end]

    # 배치 텍스트 구성
    batch_texts = [
        f"[하위분류] {row['하위분류']}\n[문의내용] {row['문의내용']}"
        for _, row in batch_df.iterrows()
    ]

    # 한 번에 임베딩 호출 (여기가 핵심!)
    batch_embeds = get_embeddings_batch(batch_texts)

    # Qdrant 포인트 생성
    points = []
    for (idx, row), emb in zip(batch_df.iterrows(), batch_embeds):
        p = PointStruct(
            id=int(idx),
            vector=emb,
            payload={
                "하위분류": row["하위분류"],
                "문의내용": row["문의내용"],
                "답변부서": row["답변부서"],
            },
        )
        points.append(p)

    # 배치 업서트
    qdrant.upsert(
        collection_name="민원데이터",
        points=points,
    )

    print(f"{end}/{n} 까지 업로드 완료")

print("전체 업로드 완료!")

128/716 까지 업로드 완료
256/716 까지 업로드 완료
384/716 까지 업로드 완료
512/716 까지 업로드 완료
640/716 까지 업로드 완료
716/716 까지 업로드 완료
전체 업로드 완료!


In [7]:
def rag_answer(question: str) -> str:
    """
    한 번 질문 -> 한 번 답변 반환 (스트림릿/웹에서 사용하기 좋게)
    """
    question = question.strip()
    if not question:
        return "질문이 비어 있어요. 뭘 물어볼지 입력해 주세요 🙂"

    # 1️⃣ 쿼리 임베딩
    q_emb = get_embeddings_batch([question])
    if not q_emb:
        return "임베딩을 생성하지 못했어요. 질문을 다시 한번 입력해 주세요."

    # 2️⃣ Qdrant에서 검색
    results = qdrant.query_points(
        collection_name="민원데이터",
        query=q_emb[0],
        limit=5,
        with_payload=True
    )

    return results

In [8]:
results = rag_answer("집가고싶다")

In [9]:
results.points[0].payload.get('답변부서')

'지진안전관리실'

In [12]:
for point in results.points:
    print(point.payload)

{'하위분류': '시설물 내진성능평가 및 내진보강 관련 문의', '문의내용': '안녕하십니까제주개발공사에서 운영하고 있는 소규모 임대주택 내진성능평가 용역을 수행하고자 계획 중에 있습니다. 해당 주택의 내진성능평가 대가 산정방식에 대해 문의드리고자 합니다. A동(연면적 659m2), B동(327m2), C동(520m2)을 내진성능평가를 실시하고자 합니다. A동과 B동의 거리는 5.0km 이고 차량으로 16분 소요되고, B동과 C동의 거리는 2.4km 이고, 차량으로 6분 소요되는 상황입니다. (참고로 A-B-C, 지도상 일직선상에 놓여 있습니다)민원답변에서 군집시설물에 대해 찾다보니 군집시설물은 점검, 진단시 현장이동, 업무협의, 보고서 작성 및 중간보고 등의 업무 등에 대해 중복되는 부분에서 기준인원수를 조정하도록 한 부분이라고 회신된 내용을 보았습니다.(답변일자 2018. 03. 12., 시설성능연구소 정일원) 도보상 이동은 어려우나, 해당 업무를 수행함에 있어 동일한 인력으로 수행시 중복되는 인력이 존재할 것이라고 보여집니다. 군집시설물로 볼 수 있는지 여부가 궁금하여 질문을 남깁니다. 회신 부탁드립니다.', '답변부서': '지진안전관리실'}
{'하위분류': '안전관리계획서 작성 관련 문의', '문의내용': '안녕하세요 고생 많으십니다!정기안전점검 항목이지보공2회, 동바리2회, 합벽2회라고 가정하겠습니다.직접인건비에 관한 질의입니다!내업, 외업을 어떻게 나누며, 회당 비용이 천차만별인데 그냥 초급 1명 넣어놓으면 시공사 입장에서 회당 비용 적어서좋은 입장인데 관련된 근거가 있나요? 그냥 직접 인건비에 안전관리계획서 작성하시는 분 마음대로 초급 1명 넣어서 점검 회 당 비용 50만원 정도에 해도 되는 건가요? 그렇게 되면 위에 가정 한대로총 6회를 300만원에 하게 되는데 여기저기 너무 천차만별 의견도 가지가지여서 확실한 답을 듣고 싶어 질의합니다. 뚜렷한 기준이 없어 난감한 상황이 많습니다. 여러 회사의 안전관리계획서 첨부드립니다.', '답변부서': '건

In [29]:
def choose_department_with_llm(query_text: str, hits) -> str:
    """유사 민원들을 보고 LLM이 담당 부서를 한 개 고르도록"""

    examples_text = ""
    for h in hits.points:
        p = h.payload
        examples_text += (
            f"- 문의내용: {p['문의내용']}\n"
            f"  부서: {p['답변부서']}\n\n"
        )

    prompt = f"""
너는 민원 라우팅 보조 시스템이다.

아래는 과거 민원 문의내용과 실제로 배정된 담당 부서 목록이다.

[과거 민원 사례]
{examples_text}

위 사례들을 참고해서, 아래 '새 민원'에 대해 가장 적절한 담당 부서 하나를 골라라.
반드시 위 사례에서 등장한 부서명 중 하나만 고르거나, 적절한 부서가 없을 경우 '추가민원'을 출력하라.
그리고 그 이유를 작성하라.

[새 민원]
\"\"\"{query_text}\"\"\"
"""

    resp = chat_client.chat.completions.create(
        model="solar-pro2",
        messages=[{"role": "user", "content": prompt}],
    )
    department = resp.choices[0].message.content.strip()
    return department

In [30]:
question = "집가고싶다"
print(choose_department_with_llm(question, rag_answer(question)))

**담당 부서**: 추가민원  

**이유**:  
제공된 '새 민원' 내용인 **"집가고싶다"**는 구체적인 업무 범위나 문의 사항이 전혀 드러나지 않는 모호한 표현입니다. 과거 민원 사례들은 모두 특정 분야(건설 안전, 지진 관리, 시설 성능 평가, 사고 조사 등)의 전문적 문의나 문제 해결을 요구했으나, 이번 민원은 단순한 감정적 표현으로 어떤 부서의 관할인지 판단할 수 있는 근거가 없습니다.  

따라서, **"추가민원"**으로 분류하여 추가 정보(구체적인 문의 내용, 관련 시설물 또는 업무 분야 등)를 요청한 후 적절한 부서로 재라우팅해야 합니다.
